# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    #Write your code here
    
    total_invested_amount = P['Amount Invested'].sum()
    
    P['Total Weight of Investment'] = P['Amount Invested'] / total_invested_amount
    
    exp_return = (rf + P['Beta'] * mkt_rp).dot(P['Total Weight of Investment'])
    
    return exp_return

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [5]:
def volatility(P, cor):
    #Write your code here
    
    total_invested_amount = P['Amount Invested'].sum()
    
    portfolio_volatility = 0
    
    for idx, row in P.iterrows():
        stock_volatility = row['Volatility']
        weight = row['Amount Invested'] / total_invested_amount
        portfolio_volatility += (weight**2) * (stock_volatility**2)
    
    additional_amount = len(P) * cor * np.prod(P.apply(lambda row: row['Volatility'] * (row['Amount Invested'] / total_invested_amount), axis=1))

    portfolio_volatility = np.sqrt(portfolio_volatility + additional_amount)
    
    return portfolio_volatility

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [7]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    #Write your code here
    return (rf + (vol / mkt_vol * mkt_rp))


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [9]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
   
    returns_data_df = pd.read_csv(return_path)

    returns_data_df['month'] = returns_data_df['month'].astype(str).str.zfill(2)
    
    returns_data_df['year'] = returns_data_df['year'].astype(str)
    
    returns_data_df['date'] = returns_data_df['year'] + returns_data_df['month']
    
    returns_data_df['date'] = returns_data_df['date'].astype(int) 
    
    factors_data_df = pd.read_csv(factors_path)

    merged_df = pd.merge(returns_data_df, factors_data_df, left_on='date', right_on='date')
    
    merged_df[['Mkt-RF', 'RF']] /= 100 

    expected_returns = merged_df.groupby('CUSIP')['RET'].mean().reset_index(name='EXP_RETURN')

    grouped = merged_df.groupby('CUSIP')

    def calculate_beta(group):
        cov_matrix = np.cov(group['RET'] - group['RF'], group['Mkt-RF'])
        beta = cov_matrix[0, 1] / cov_matrix[1, 1]
        return beta

    betas = grouped.apply(calculate_beta).reset_index(name='BETA')

    returns_data_df = pd.merge(expected_returns, betas, on='CUSIP')

    output = pd.DataFrame({
        "CUSIP": returns_data_df['CUSIP'].values,
        "EXP_RETURN": returns_data_df['EXP_RETURN'].values,
        "BETA": returns_data_df['BETA'].values,
    })

    return output

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [11]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    slope_for_security = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    return slope_for_security[1]